<a href="https://colab.research.google.com/github/winterForestStump/thesis/blob/main/retrieval/Retrievals_reranked_l2_evaluation_phi3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#  LLAMA_CUBLAS is deprecated and will be removed in the future. Use LLAMA_CUDA instead

In [9]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --verbose

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 MB 15.6 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.1/152.1 kB 4.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 7.3 MB/s eta 0:00:00
    Using cached pathspec-0.12.1-py3-none-any.whl (31 kB)
  ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
  langchain-core 0.2.4 requires packaging<24.0,>=23.2, but you have packaging 24.0 which is incompatible.
  Installing build dependencies ... done
  Running command Getting requirements to build wheel
  Could not determine CMake version via --version, got '' 'Traceback (most recent call last)

In [4]:
!huggingface-cli download microsoft/Phi-3-mini-4k-instruct-gguf Phi-3-mini-4k-instruct-fp16.gguf --local-dir ./models --local-dir-use-symlinks False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/download.py:132: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
Phi-3-mini-4k-instruct-fp16.gguf: 100% 7.64G/7.64G [00:58<00:00, 131MB/s]
Download complete. Moving file to models/Phi-3-mini-4k-instruct-fp16.gguf
models/Phi-3-mini-4k-instruct-fp16.gguf


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!pip install langchain langchain-core langchain-community --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.6/973.6 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.4/310.4 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.8/124.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 11.3 MB/s eta 0:00:00


In [10]:
from langchain_community.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp
from langchain_core.output_parsers import JsonOutputParser
from json import JSONDecodeError
from langchain_core.exceptions import OutputParserException
import pandas as pd
import os

In [11]:
TEMP = 0
N_CTX = 4096
N_GPU_L = -1

llm = LlamaCpp(
    model_path="/content/models/Phi-3-mini-4k-instruct-fp16.gguf",
    temperature=TEMP,
    n_ctx=N_CTX,
    n_gpu_layers = N_GPU_L,
    verbose=True
)

llama_model_loader: loaded meta data with 23 key-value pairs and 195 tensors from /content/models/Phi-3-mini-4k-instruct-fp16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                  phi3.attention.head_count u32              = 32
llama_model_loader: - kv   7:               phi3.attention.head_count

In [12]:
### Retrieval Grader for approach 1

prompt_1 = PromptTemplate(
    template="""
    <|assistant|> You are a grader assessing relevance of a retrieved document to a user question. If the document contains keywords related to the user question, grade it as relevant.
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explaination. <|end|>
    <|user|> Here is the retrieved document: \n\n {document} \n\n Here is the user question: {question} <|end|>
    <|assistant|>
    """,
    input_variables=["question", "document"],
)

retrieval_grader_1 = prompt_1 | llm | JsonOutputParser()

In [16]:
folder_path = '/content/drive/MyDrive/Thesis/retrievals/reranked/'

dataframes = []
for file_name in os.listdir(folder_path):
    if file_name.endswith('.json'):
        approach_name = file_name
        dataframes.append(approach_name)

In [60]:
def calculate_results():
  results_list = []

  for i in range(len(dataframes)):

    approach = pd.read_json(os.path.join(folder_path + dataframes[i]))
    try:
      for j in range(len(approach)):
        question = approach['question'][j]
        list_of_lists = approach['context'][j]
        doc_txt = [inner_list[1] for inner_list in list_of_lists if len(inner_list)>1]
        if not doc_txt:
          continue
        try:
          answer = retrieval_grader_1.invoke({"question": question, "document": [doc_txt]})
          results_list.append(pd.DataFrame({"question": question, "answer": [answer], "document": [doc_txt]}))
        except JSONDecodeError as e:
          print(f"JSONDecodeError occurred for question: {question}. Skipping...")
          continue
        except OutputParserException as e:
          print(f"OutputParserException occurred for question: {question}. Skipping...")
          continue
    except KeyError as e:
      print(f"KeyError occurred: {e}. Skipping approach for dataframe {dataframes[i]}")
      continue


  if results_list:
    results = pd.concat(results_list, ignore_index=True)
    results.to_json(f'/content/drive/MyDrive/Thesis/evaluation/eval_reranked.json')
    score = 0
    for i in range(len(results)):
      if (results['answer'][i] is not None) and (results['answer'][i]['score'] == 'yes'):
        score += 1
    total_score = score/len(results)
    print(f'Total score is {total_score}')
  else:
    print(f"No results to concatenate")

In [61]:
calculate_results()

Llama.generate: prefix-match hit

llama_print_timings:        load time =     430.15 ms
llama_print_timings:      sample time =      68.83 ms /   120 runs   (    0.57 ms per token,  1743.45 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =    4768.47 ms /   120 runs   (   39.74 ms per token,    25.17 tokens per second)
llama_print_timings:       total time =    4926.60 ms /   120 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     430.15 ms
llama_print_timings:      sample time =      38.06 ms /    67 runs   (    0.57 ms per token,  1760.29 tokens per second)
llama_print_timings: prompt eval time =    8678.85 ms /  1549 tokens (    5.60 ms per token,   178.48 tokens per second)
llama_print_timings:        eval time =    2704.11 ms /    66 runs   (   40.97 ms per token,    24.41 tokens per second)
llama_print_timings:       to

OutputParserException occurred for question: What is the company's cost of goods sold (COGS) and how does the COGS compare to the total revenue?. Skipping...



llama_print_timings:        load time =     430.15 ms
llama_print_timings:      sample time =     108.83 ms /   210 runs   (    0.52 ms per token,  1929.54 tokens per second)
llama_print_timings: prompt eval time =    5829.35 ms /  1075 tokens (    5.42 ms per token,   184.41 tokens per second)
llama_print_timings:        eval time =    8331.92 ms /   209 runs   (   39.87 ms per token,    25.08 tokens per second)
llama_print_timings:       total time =   14429.50 ms /  1284 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     430.15 ms
llama_print_timings:      sample time =      48.49 ms /    83 runs   (    0.58 ms per token,  1711.55 tokens per second)
llama_print_timings: prompt eval time =    4466.91 ms /   839 tokens (    5.32 ms per token,   187.83 tokens per second)
llama_print_timings:        eval time =    3227.13 ms /    82 runs   (   39.36 ms per token,    25.41 tokens per second)
llama_print_timings:       total time =    7811.25 ms /   921 

OutputParserException occurred for question: What is the company's net income for the current fiscal year and how has net income trended over the past few years?. Skipping...



llama_print_timings:        load time =     430.15 ms
llama_print_timings:      sample time =     143.66 ms /   256 runs   (    0.56 ms per token,  1782.00 tokens per second)
llama_print_timings: prompt eval time =    4581.83 ms /   842 tokens (    5.44 ms per token,   183.77 tokens per second)
llama_print_timings:        eval time =   10297.47 ms /   255 runs   (   40.38 ms per token,    24.76 tokens per second)
llama_print_timings:       total time =   15222.82 ms /  1097 tokens
Llama.generate: prefix-match hit


...

OutputParserException occurred for question: What is the company's cost of goods sold (COGS) and how does the COGS compare to the total revenue?. Skipping...



llama_print_timings:        load time =     430.15 ms
llama_print_timings:      sample time =      43.59 ms /    64 runs   (    0.68 ms per token,  1468.36 tokens per second)
llama_print_timings: prompt eval time =    4426.42 ms /   824 tokens (    5.37 ms per token,   186.15 tokens per second)
llama_print_timings:        eval time =    2565.01 ms /    64 runs   (   40.08 ms per token,    24.95 tokens per second)
llama_print_timings:       total time =    7097.72 ms /   888 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     430.15 ms
llama_print_timings:      sample time =      47.32 ms /    83 runs   (    0.57 ms per token,  1753.98 tokens per second)
llama_print_timings: prompt eval time =    5993.49 ms /  1096 tokens (    5.47 ms per token,   182.86 tokens per second)
llama_print_timings:        eval time =    3330.21 ms /    82 runs   (   40.61 ms per token,    24.62 tokens per second)
llama_print_timings:       total time =    9438.33 ms /  1178 

OutputParserException occurred for question: What is the company's geographic breakdown of revenue and are there any notable trends or shifts?. Skipping...



llama_print_timings:        load time =     430.15 ms
llama_print_timings:      sample time =      73.69 ms /   123 runs   (    0.60 ms per token,  1669.09 tokens per second)
llama_print_timings: prompt eval time =    3635.78 ms /   687 tokens (    5.29 ms per token,   188.96 tokens per second)
llama_print_timings:        eval time =    4830.58 ms /   122 runs   (   39.59 ms per token,    25.26 tokens per second)
llama_print_timings:       total time =    8627.59 ms /   809 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     430.15 ms
llama_print_timings:      sample time =      18.07 ms /    32 runs   (    0.56 ms per token,  1771.38 tokens per second)
llama_print_timings: prompt eval time =    4572.56 ms /   848 tokens (    5.39 ms per token,   185.45 tokens per second)
llama_print_timings:        eval time =    1272.54 ms /    32 runs   (   39.77 ms per token,    25.15 tokens per second)
llama_print_timings:       total time =    5891.38 ms /   880 

TypeError: list indices must be integers or slices, not str